In [1]:
import time
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from sklearn_extra.cluster import KMedoids

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.mixture import GaussianMixture

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import MeanShift
from sklearn.cluster import AgglomerativeClustering
# from konlpy.tag import Komoran

from pandas_datareader import data as pdr
import yfinance as yfin
from datetime import datetime

from scipy.cluster.hierarchy import linkage 
from scipy.cluster.hierarchy import dendrogram 

from tslearn.clustering import TimeSeriesKMeans
from tslearn.metrics import dtw

import scipy.spatial.distance as ssd

In [5]:
# 크롤링(종목코드, 종목명)

url = "https://finance.naver.com/sise/entryJongmok.naver?&page={}"
name_lists= []
code_lists= []
for j in range(1,21):
    
    r = requests.get(url.format(j), headers = {'User-agent':'Mozilla/5.0'})
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    x = page.select('td[class="ctg"]')

    for i in range(0,10):
        code = x[i].find("a")["href"][22:]
        code_lists.append(code)
        
        name = page.select('td[class="ctg"]')[i].a.text
        name_lists.append(name)
    # time.sleep(2)


In [6]:
df = pd.DataFrame({"name": name_lists, "code": code_lists})
df

,name,code
0,삼성전자,005930
1,SK하이닉스,000660
2,LG에너지솔루션,373220
3,현대차,005380
4,삼성바이오로직스,207940
...,...,...
195,롯데관광개발,032350
196,신풍제약,019170
197,KG스틸,016380
198,영풍,000670


In [ ]:
#크롤링(기업개요)

url = 'https://finance.naver.com/item/coinfo.naver?code={}'
x1_list = []
x2_list = []
x3_list = []
for i in code_lists:
    r = requests.get(url.format(i), headers = {'User-agent':'Mozilla/5.0'})
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    x1 = page.select('p')[0].text
    x1_list.append(x1)
    x2 = page.select('p')[1].text
    x2_list.append(x2)
    x3 = page.select('p')[2].text
    x3_list.append(x3)
df2['x1'] = x1_list
df2['x2'] = x2_list
df2['x3'] = x3_list
df2

In [ ]:
#기업개요 하나로 묶은 x 컬럼 생성
x = []
for i in range(0,200):
    x.append(df.iloc[i]["x1"]+" "+df.iloc[i]["x2"]+" "+df.iloc[i]["x3"])
df["x"] = x
df

In [ ]:
# 크롤링 (업종)
url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}"
labels_kse = []
labels_fics = []

for j in range(0,200):
    
    r = requests.get(url.format(df.iloc[j]["code"]))
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    y = page.select('span')[4].text
    y2 = page.select('span')[7].text
    y = y.replace('\xa0',' ')
    y2 = y2.replace('\xa0',' ')
    y = y[9:]
    y2 = y2[6:]
    y,y2
    labels_kse.append(y)
    labels_fics.append(y2)
    time.sleep(0.5)
    

df["labels_kse"] = labels_kse
df["labels_fics"] = labels_fics

In [ ]:
# 크롤링 (PER)
url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}"
per = []

for j in range(0,200):
    r = requests.get(url.format(df_origin.iloc[j]["code"]))
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    y = page.select('dd')[4].text
    try :
        per.append(float(y))
    except :
        y = None
        per.append(y)

df["PER"] = per


In [ ]:
# 크롤링 (PBR)
url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}"
pbr = []

for j in range(0,200):
    r = requests.get(url.format(df_origin.iloc[j]["code"]))
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    y = page.select('dd')[10].text
    try :
        pbr.append(float(y))
    except :
        y = None
        pbr.append(y)

df["PBR"] = pbr

In [ ]:
# 크롤링 (Dividend)
url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}"
pbr = []

for j in range(0,200):
    r = requests.get(url.format(df_origin.iloc[j]["code"]))
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    y = page.select('dd')[12].text[:-1]
    try :
        pbr.append(float(y)/100)
    except :
        y = None
        pbr.append(y)

df["Dividend"] = pbr

In [ ]:
# 크롤링 (EV/EBITDA)
url = "https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={}&target=finsum_more"
pbr = []

for j in range(0,200):
    r = requests.get(url.format(df_origin.iloc[j]["code"]), headers = {'User-agent':'Mozilla/5.0'})
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    y = page.select("td[class='num']")[11].text
    try :
        pbr.append(float(y))
    except :
        y = None
        pbr.append(y)

df["EV/EBITDA"] = pbr

In [5]:
# 크롤링 (PCR)
url = "https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={}&target=finsum_more"
pbr = []

r = requests.get(url.format('005930'), headers = {'User-agent':'Mozilla/5.0'})
html = r.text
page = BeautifulSoup(html, 'html.parser')
y = page.select("td[class='num']")[10].text
y

'7.67'

In [7]:
df.iloc[0]["code"]

'005930'

In [8]:
# 크롤링 (시가, 종가, 최고가, 최저가)
df_price = pd.DataFrame()
yfin.pdr_override()


code = "{}.KS"
start = datetime(2021,4,1)
end = datetime(2024,4,1)

for i in range(0,200):
    df_price[df.iloc[i]["name"]] = pdr.get_data_yahoo(code.format(df.iloc[i]["code"]), start, end)["Close"].copy()
#     time.sleep(1)
df_price



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

<ipython-input-8-835fd22996db>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_price[df.iloc[i]["name"]] = pdr.get_data_yahoo(code.format(df.iloc[i]["code"]), start, end)["Close"].copy()


[*********************100%***********************]  1 of 1 completed


<ipython-input-8-835fd22996db>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_price[df.iloc[i]["name"]] = pdr.get_data_yahoo(code.format(df.iloc[i]["code"]), start, end)["Close"].copy()


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


<ipython-input-8-835fd22996db>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_price[df.iloc[i]["name"]] = pdr.get_data_yahoo(code.format(df.iloc[i]["code"]), start, end)["Close"].copy()


[*********************100%***********************]  1 of 1 completed


<ipython-input-8-835fd22996db>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_price[df.iloc[i]["name"]] = pdr.get_data_yahoo(code.format(df.iloc[i]["code"]), start, end)["Close"].copy()


,삼성전자,SK하이닉스,LG에너지솔루션,현대차,삼성바이오로직스,기아,셀트리온,POSCO홀딩스,KB금융,NAVER,...,후성,세아베스틸지주,아이에스동서,TKG휴켐스,명신산업,롯데관광개발,신풍제약,KG스틸,영풍,녹십자홀딩스
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-01,82900.0,140500.0,NaN,219000.0,739153.8750,83500.0,305901.21875,318500.0,55000.0,378500.0,...,10550.0,18950.0,56500.0,22200.0,25650.0,19500.0,82900.0,14550.0,582000.0,34700.0
2021-04-02,84800.0,141000.0,NaN,233500.0,736185.3750,86600.0,302130.46875,318500.0,53900.0,379500.0,...,10450.0,18300.0,56000.0,22050.0,26200.0,19300.0,81300.0,14150.0,581000.0,34400.0
2021-04-05,85400.0,143000.0,NaN,234000.0,742122.3750,85500.0,290346.90625,334500.0,54600.0,377000.0,...,10150.0,18750.0,56400.0,22250.0,27900.0,19050.0,85300.0,14600.0,588000.0,33750.0
2021-04-06,86000.0,143000.0,NaN,230000.0,746080.3750,84900.0,286104.81250,328500.0,53000.0,388500.0,...,10250.0,18400.0,56400.0,22100.0,27600.0,19950.0,84200.0,14400.0,591000.0,32550.0
2021-04-07,85600.0,143500.0,NaN,230000.0,756964.8125,85300.0,288461.53125,329000.0,53600.0,384500.0,...,10200.0,19000.0,55600.0,22400.0,27200.0,19950.0,85900.0,15800.0,589000.0,32950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-25,78200.0,169400.0,414500.0,239500.0,840000.0000,111500.0,188900.00000,432000.0,72000.0,188000.0,...,7940.0,23100.0,29850.0,19950.0,15460.0,9150.0,18320.0,7000.0,475000.0,17790.0
2024-03-26,79900.0,176600.0,406000.0,240500.0,828000.0000,114400.0,186400.00000,429000.0,71800.0,188900.0,...,7900.0,23350.0,29600.0,19960.0,15460.0,9450.0,16820.0,7150.0,476500.0,17800.0
2024-03-27,79800.0,181200.0,409000.0,244000.0,836000.0000,113600.0,184200.00000,428000.0,69000.0,189300.0,...,7980.0,22900.0,29250.0,19960.0,15820.0,9990.0,16150.0,7150.0,479500.0,17680.0


In [9]:
df_price
df_price.to_pickle('C:/Users/Dohyung/Desktop/ssafy/Github/PJT-Clustering-KOSPI200/fkl/price-2024.fkl')

In [27]:
df_price = pd.read_pickle('C:/Users/Dohyung/Desktop/ssafy/Github/PJT-Clustering-KOSPI200/fkl/price-2024.fkl')
df_price = df_price .transpose()
df_price = df_price .fillna(0.0)
df_price .to_csv('./price-2024.csv',encoding="utf-8-sig")

In [28]:
df_price 

Date,2021-04-01,2021-04-02,2021-04-05,2021-04-06,2021-04-07,2021-04-08,2021-04-09,2021-04-12,2021-04-13,2021-04-14,...,2024-03-18,2024-03-19,2024-03-20,2024-03-21,2024-03-22,2024-03-25,2024-03-26,2024-03-27,2024-03-28,2024-03-29
삼성전자,82900.000,84800.000,85400.000,86000.000,85600.0000,84700.0000,83600.0000,83200.00,84000.00,84000.00,...,72800.0,72800.0,76900.0,79300.0,78900.0,78200.0,79900.0,79800.0,80800.0,82400.0
SK하이닉스,140500.000,141000.000,143000.000,143000.000,143500.0000,144000.0000,140000.0000,137500.00,139500.00,137000.00,...,164300.0,160200.0,156500.0,170000.0,169800.0,169400.0,176600.0,181200.0,178200.0,183000.0
LG에너지솔루션,0.000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.00,0.00,0.00,...,407500.0,403000.0,407500.0,413500.0,413500.0,414500.0,406000.0,409000.0,401500.0,395000.0
현대차,219000.000,233500.000,234000.000,230000.000,230000.0000,231500.0000,228500.0000,226000.00,230000.00,232500.00,...,242500.0,237500.0,241000.0,252000.0,243500.0,239500.0,240500.0,244000.0,237000.0,233000.0
삼성바이오로직스,739153.875,736185.375,742122.375,746080.375,756964.8125,754985.8125,761912.3125,773786.25,775765.25,781702.25,...,859000.0,848000.0,850000.0,857000.0,840000.0,840000.0,828000.0,836000.0,833000.0,831000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
롯데관광개발,19500.000,19300.000,19050.000,19950.000,19950.0000,19500.0000,19300.0000,18900.00,18900.00,18950.00,...,9030.0,9140.0,8970.0,9020.0,9190.0,9150.0,9450.0,9990.0,9750.0,9790.0
신풍제약,82900.000,81300.000,85300.000,84200.000,85900.0000,88900.0000,89100.0000,90800.00,90100.00,90000.00,...,11740.0,11810.0,12770.0,12670.0,16260.0,18320.0,16820.0,16150.0,15820.0,15510.0
KG스틸,14550.000,14150.000,14600.000,14400.000,15800.0000,16100.0000,15600.0000,15700.00,15250.00,14900.00,...,7200.0,7040.0,7000.0,7070.0,7090.0,7000.0,7150.0,7150.0,7020.0,7000.0
영풍,582000.000,581000.000,588000.000,591000.000,589000.0000,589000.0000,589000.0000,606000.00,603000.00,612000.00,...,470500.0,468000.0,466500.0,481500.0,483000.0,475000.0,476500.0,479500.0,477000.0,472000.0


In [25]:
# price data 를 등락률로 변환
df_updown = pd.DataFrame(index=df_price.index, columns=df_price.columns)
for i in range (1,738):
    for j in range(0,200):
        try :
            gap = df_price.iloc[i][j]-df_price.iloc[i-1][j]
            df_updown.iloc[i][j] = gap/df_price.iloc[i-1][j]
        except:
            pass

<ipython-input-25-0b0f49f4362c>:7: RuntimeWarning: divide by zero encountered in double_scalars
  df_updown.iloc[i][j] = gap/df_price.iloc[i-1][j]
<ipython-input-25-0b0f49f4362c>:7: RuntimeWarning: invalid value encountered in double_scalars
  df_updown.iloc[i][j] = gap/df_price.iloc[i-1][j]


In [26]:
df_updown

Date,2021-04-01,2021-04-02,2021-04-05,2021-04-06,2021-04-07,2021-04-08,2021-04-09,2021-04-12,2021-04-13,2021-04-14,...,2024-03-18,2024-03-19,2024-03-20,2024-03-21,2024-03-22,2024-03-25,2024-03-26,2024-03-27,2024-03-28,2024-03-29
삼성전자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SK하이닉스,0.694813,0.662736,0.674473,0.662791,0.676402,0.700118,0.674641,0.652644,0.660714,0.630952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LG에너지솔루션,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
현대차,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
삼성바이오로직스,2.375132,2.152828,2.171463,2.243828,2.291151,2.261278,2.334408,2.423833,2.372892,2.36216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
롯데관광개발,-0.239766,-0.263359,-0.317204,-0.277174,-0.266544,-0.271028,-0.281192,-0.292135,-0.341463,-0.346552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
신풍제약,3.251282,3.212435,3.47769,3.220551,3.305764,3.558974,3.61658,3.804233,3.767196,3.74934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KG스틸,-0.824487,-0.825953,-0.828839,-0.828979,-0.816065,-0.818898,-0.824916,-0.827093,-0.830744,-0.834444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
영풍,39.0,40.060071,39.273973,40.041667,36.278481,35.583851,36.75641,37.598726,38.540984,40.073826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
import pandas as pd
df_price = pd.read_pickle('../../fkl/price_normal.fkl')
df_price = df_price.dropna(axis = 0)
df_price

Date,2021-04-01,2021-04-02,2021-04-05,2021-04-06,2021-04-07,2021-04-08,2021-04-09,2021-04-12,2021-04-13,2021-04-14,...,2024-03-18,2024-03-19,2024-03-20,2024-03-21,2024-03-22,2024-03-25,2024-03-26,2024-03-27,2024-03-28,2024-03-29
삼성전자,-0.151859,-0.141719,-0.138566,-0.133920,-0.138996,-0.146139,-0.154314,-0.158491,-0.158405,-0.160588,...,-0.197813,-0.192534,-0.158344,-0.148769,-0.152027,-0.157544,-0.144838,-0.147288,-0.134569,-0.115597
SK하이닉스,0.170613,0.173076,0.186036,0.187680,0.186812,0.183779,0.158961,0.143069,0.146274,0.128679,...,0.604030,0.580947,0.544400,0.640603,0.640917,0.632711,0.700086,0.730686,0.714299,0.765605
LG에너지솔루션,-0.615973,-0.616712,-0.619834,-0.619141,-0.620675,-0.617370,-0.618672,-0.620550,-0.619540,-0.619050,...,2.735266,2.729701,2.760339,2.759809,2.766776,2.756524,2.704487,2.703097,2.660421,2.622611
현대차,0.610094,0.691200,0.698863,0.678544,0.673555,0.670588,0.650536,0.634562,0.643092,0.649907,...,1.289320,1.265043,1.290403,1.354257,1.283821,1.240134,1.258417,1.274441,1.226757,1.203578
삼성바이오로직스,3.522166,3.506908,3.562365,3.590323,3.638832,3.583017,3.613385,3.676742,3.639182,3.647386,...,6.691900,6.667903,6.666925,6.619637,6.487246,6.443517,6.391745,6.400285,6.421065,6.441737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
롯데관광개발,-0.506803,-0.508606,-0.512479,-0.506581,-0.508415,-0.508881,-0.511470,-0.515587,-0.515785,-0.515623,...,-0.756649,-0.755918,-0.758060,-0.760423,-0.760126,-0.755868,-0.760401,-0.751740,-0.753790,-0.751622
신풍제약,-0.151859,-0.161323,-0.139130,-0.144075,-0.137308,-0.122772,-0.123764,-0.116284,-0.124918,-0.127841,...,-0.732901,-0.732289,-0.724512,-0.728657,-0.698452,-0.676410,-0.696005,-0.698403,-0.700888,-0.701518
KG스틸,-0.534515,-0.537453,-0.537556,-0.537894,-0.531767,-0.527797,-0.532021,-0.533358,-0.535822,-0.537728,...,-0.772686,-0.774503,-0.775452,-0.777394,-0.778444,-0.774498,-0.780497,-0.776330,-0.777583,-0.776061
영풍,2.642343,2.637663,2.693815,2.715344,2.693679,2.659550,2.652940,2.744926,2.690752,2.721172,...,3.287355,3.304944,3.281217,3.351620,3.373042,3.280761,3.320486,3.313522,3.318425,3.297090


In [18]:
from dtaidistance import dtw
dtw_matrix = dtw.distance_matrix(df_price.to_numpy(), show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████| 194/194 [32:03<00:00,  9.91s/it]


In [19]:
dtw_matrix
df_dtw_matrix = pd.DataFrame(dtw_matrix)
df_dtw_matrix.index = df_price.index


In [20]:
df_dtw_matrix

,0,1,2,3,4,5,6,7,8,9,...,184,185,186,187,188,189,190,191,192,193
삼성전자,0.000000,2.869467,7.750365,7.998594,5.084879,7.080470,7.049312,3.760821,5.806505,6.108392,...,9.234514,5.142929,7.526803,7.147105,5.187178,14.580965,7.027403,5.022545,17.638922,22.867415
SK하이닉스,2.869467,0.000000,9.453427,9.052383,5.259753,8.613628,8.787274,4.788209,7.697084,5.142949,...,7.042576,4.740617,9.587700,5.740314,6.541818,13.296300,5.085923,4.241785,14.977939,20.482347
NAVER,7.750365,9.453427,0.000000,6.312767,8.391875,2.224759,4.463978,7.487354,4.939854,11.708098,...,10.604207,4.457010,3.184299,9.459375,3.308220,16.957284,11.236154,6.089922,22.704672,26.548097
삼성바이오로직스,7.998594,9.052383,6.312767,0.000000,8.005945,5.164912,5.424367,7.537216,6.486340,10.885658,...,9.984731,7.310370,5.443423,8.188671,5.488283,13.382969,10.323901,6.261957,16.792976,21.889155
LG화학,5.084879,5.259753,8.391875,8.005945,0.000000,8.104657,7.148895,4.213296,5.919671,7.578024,...,7.776600,5.542517,6.915840,6.345079,6.712172,14.323839,7.966059,5.534552,15.226068,20.536643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
현대두산인프라코어,14.580965,13.296300,16.957284,13.382969,14.323839,15.337753,17.271091,15.151025,17.777913,11.061849,...,9.239220,14.336278,16.487159,10.646334,15.524986,0.000000,9.857081,11.836621,13.191419,14.903657
롯데하이마트,7.027403,5.085923,11.236154,10.323901,7.966059,10.229490,11.657364,7.164353,10.661715,4.760187,...,6.166942,6.182018,11.411102,3.763640,8.627309,9.857081,0.000000,4.861798,13.180435,18.586290
LX하우시스,5.022545,4.241785,6.089922,6.261957,5.534552,5.441999,7.043210,5.253966,6.763026,5.406743,...,5.270118,4.173046,6.523484,5.108574,4.472748,11.836621,4.861798,0.000000,17.454249,22.516920
삼양식품,17.638922,14.977939,22.704672,16.792976,15.226068,21.894130,20.581945,17.924761,21.570816,8.817446,...,11.508948,19.577548,22.257496,15.260678,21.023643,13.191419,13.180435,17.454249,0.000000,5.740042


In [8]:
df_dart = pd.read_pickle('./dart_khaiii.fkl')

In [9]:
import re


for i in range(0,200):
    test = df_dart.iloc[i]['x']
    test = re.sub('[☞※+#/\?:^$@*\"※~&%ㆍ!》■○◆]',' ',test)
    test = test.replace('\xa0',' ')
    test = test.replace('\n',' ')
    test = test.replace('㈜','')
    df_dart.iloc[i]['x'] =test


KeyError: 'x'

In [12]:
df_dart.to_csv('./dart.csv', encoding= 'euc-kr')

In [ ]:
# 신규 상장 기업인듯
for company in df_price:
    if df_price[company].isnull().sum() != 0:
        print('{}, {}개'.format(company, df_price[company].isnull().sum()))

In [29]:
for company in df_price:
    print(df_price[company].mean(), df_price[company].std())
    
# 평균 빼고 표준편차로 나누기
for company in df_price:
    df_price[company] = (df_price[company][:] - df_price[company].mean()) / df_price[company].std()

110025.04494384765 178619.8560229196
110100.8551953125 178528.93786538864
109988.26137207031 177447.885146961
109735.73946289062 177238.80993089304
110301.2965209961 177711.84656236382
110967.23331054687 179741.99805186124
111381.68556396484 180033.5653145494
111738.46118164062 180063.68406246128
112854.881015625 182159.17270028676
113423.1688671875 183221.36831936747
113940.87396484375 183978.55882882772
114904.21327392578 185941.31860871596
115099.07982910157 185191.08043741173
116076.7325390625 186517.9809342294
114786.07813476563 183823.10600598846
115036.4284423828 182872.69938541652
115622.07429199219 186294.2355241864
116561.32528320313 187076.66134635077
116077.46541015625 185417.39262705066
114574.66627929687 182866.64671572694
114620.47981201172 184035.9257450934
113709.5933984375 183115.8828925457
112557.06653076172 182759.1570124647
112953.60081787109 183053.62468888215
115092.97645019532 185736.13932108722
115991.05270263672 185964.23691996693
118222.2428881836 189156.8708

In [31]:
df_price.to_pickle('../../fkl/price_normal.fkl')

In [ ]:
# 마지막 가격 기준
for company in df_price:
    df_price[company] = df_price[company][:] / df_price[company][-1] * 100

In [ ]:
#PER, Dividend 범주형으로 바꾸기
A = df_finance["PER"].copy()

for i in range(0,198):
    if A[i] == 0:
        A[i] = 0
    elif A[i] > 0 and A[i]<=2.0:
        A[i] = 1
    elif A[i] > 2 and A[i]<=4.0:
        A[i] = 2
    elif A[i] >4.0:
        A[i] = 3
A
df_finance["PER"] = A
df_finance


In [ ]:
A = df_finance["Dividend"].copy()

for i in range(0,198):
    if A[i] == 0:
        A[i] = 0
    elif A[i] < 0 and A[i]>=-5.0:
        A[i] = 1
    elif A[i] < -5.0 and A[i]>=-6.0:
        A[i] = 2
    elif A[i] < -6.0:
        A[i] = 3
A
df_finance["Dividend"] = A
df_finance